# 📓 Synthetic Data Generator


In [ ]:
# Parameters
row_count = 10000
schema_config = {
    "id": "int",
    "category": "string",
    "value": "float",
    "timestamp": "timestamp"
}
distribution = "uniform"

# Generate base dataset
df = spark.range(0, row_count).withColumn("category", ...)
df.write.format("parquet").save("/lakehouse/base/parquet_10k")

# Generate update slice
update_df = df.sample(fraction=0.01).withColumn("change_type", ...)
update_df.write.format("parquet").save("/lakehouse/updates/parquet_10k")